In [1]:
import torch
import json

from transformers import AutoModelForQuestionAnswering
from model.parameters import Params
from model.predict import predict
from utils.calculate_f1 import calculate_f1_text

d:\pr\exact_match_bert\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = Params.DEVICE

In [3]:
device

'cpu'

## Load Model

In [4]:
model = AutoModelForQuestionAnswering.from_pretrained("MilyaShams/rubert-russian-qa-sberquad").to(device)
model.load_state_dict(torch.load("checkpoints/checkpoint_3.pt", map_location=device))

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 931.96it/s, Materializing param=qa_outputs.weight]                                      


<All keys matched successfully>

## Predict test file

In [5]:
predicts = []

In [6]:
with open("data/test.json", "r", encoding="utf-8") as file:
    test_data = json.load(file)

In [7]:
for block in test_data:
    new_block = block
    answer, start_ind, end_ind = predict(
        model,
        new_block["label"],
        new_block["text"]
        )
    new_block["extracted_part"] = {
        "text": [answer],
        "answer_start": [start_ind],
        "answer_end": [end_ind]
    }
    predicts.append(new_block)

In [8]:
with open("data/ref_and_pred_data/predicts.json", "w", encoding="utf-8") as pred:
    json.dump(predicts, pred, indent=4, ensure_ascii=False)

## Comparison with reference file

In [9]:
with open("data/ref_and_pred_data/reference.json", "r", encoding="utf-8") as f:
    reference = json.load(f)

with open("data/ref_and_pred_data/predicts.json", "r", encoding="utf-8") as f:
    predicts = json.load(f)

In [10]:
count_equals = 0
total_examples = len(predicts)
average_f1 = 0
total_not_eq = 0

In [11]:
for ref, pred in zip(reference, predicts):
    extracted_r = ref["extracted_part"]
    extracted_p = pred["extracted_part"]

    if extracted_r["text"] == extracted_p["text"] and \
        extracted_r["answer_start"] == extracted_p["answer_start"] and \
        extracted_r["answer_end"] == extracted_p["answer_end"]:
        count_equals += 1
    else:
        print("="*50)
        print(extracted_r["text"])
        print(extracted_p["text"])
        f1 = calculate_f1_text(extracted_p["text"], extracted_r["text"])
        print(f1)
        average_f1 += f1
        total_not_eq += 1

['Размер обеспечения исполнения Контракта составляет 10 процентов цены контракта, что составляет ____________ (________________)']
['Размер обеспечения исполнения Контракта составляет 10 процентов цены контракта, что составляет ____________ (________________) рублей.']
0.9545454545454545
['Сумма']
['Сумма обеспечения исполнения Контракта устанавливается в размере 5% от цены Контракта']
0.32
['Сумма']
['Сумма обеспечения исполнения Контракта устанавливается в размере 5% от цены Контракта']
0.32
['Поставщик при заключении Контракта должен предоставить Заказчику обеспечение исполнения Контракта в размере 5 % от цены контракта']
['Поставщик при заключении Контракта должен предоставить Заказчику обеспечение исполнения Контракта в размере 5 % от цены контракта, что составляет _____________ () рублей ___ коп.']
0.9811320754716981
['Обеспечение исполнения Контракта устанавливается в размере 10% от цены Контракта, в размере ____________ (___________) руб. __ коп.']
['Обеспечение исполнения Конт

## Results

In [12]:
result = count_equals / total_examples

In [13]:
result

0.8050314465408805

In [14]:
f1 = average_f1 / total_not_eq

In [15]:
f1

0.7854038024817661